# Generating SHEPHARD files of protein stability
###### Last updated 2022-12-08

### TL/DR
This notebook walks through how to convert the CSV files from Tsuboyama *et al.* to SHEPHARD compliant Track files. Please make sure you understand what these data and where data quality/filtering needs to be considered before using the resulting SHEPHARD files.

## Background
[SHEPHARD](https://shephard.readthedocs.io/) is a Python-based analysis framework for working with protien annotation data. For more details, our [recent preprint](https://www.biorxiv.org/content/10.1101/2022.09.18.508433v1) is only ~3 pages long and was written to be as accessible as possible even if you're not a computational person. The bottom line is that it provides a standard "Pythonic" way to interact with different types of protein annotations.

When the [Tsuboyama *et al.*](https://www.biorxiv.org/content/10.1101/2022.12.06.519132v3) preprint came out, we were excited to explore the data, and parsing it into SHEPHARD felt like (to us) the easiest way to jumpstart that process. 

This notebook provides the code for generating those annotations from the raw `.csv` data from the paper. We're sharing the data here now in case it is of use to anyone else. The original data [are available here](https://zenodo.org/record/7401275), and we checked with Gabriel and Kotaro before posting this to make sure they were OK with sharing these data in an alternative format.

## About
This Jupyter notebook takes the `parse_K50_dG_Dataset1_Dataset2.csv` data from *Tsuboyama et al* (from [https://zenodo.org/record/7401275](https://zenodo.org/record/7401275))  and generates SHEPHARD-compliant track files for the following six datasets:

1. `deltaG_t` (ΔG calculated from median of posteriors of K50 trypsin (kcal/mol))

2. `deltaG_c` (ΔG calculated from median of posteriors of K50 chymotrypsin (kcal/mol))

3. `deltaG` (ΔG calculated from median of posteriors of K50 trypsin + chymotrypsin (kcal/mol))

4. `K50T` (Median of posteriors of K50 trypsin in log10 scale (μM))

5. `K50C` (Median of posteriors of K50 chymotrypsin in log10 scale (μM))

And for each of these the associated confidence intervals (CI) which are calculated as top 2.5 percentile minus top 97.5 percentile to give the range of the deltaG distributions (as calculated based on the K50s).

If this sounds confusing, **please** read the paper before using the data !!!

### Track-file format
The data here are large-scale mutagenesis data, whereby for many of the proteins we have deep saturation mutagenesis (i.e. each residue mutated to every other residue). To keep things simple, the SHEPHARD files reflect data from constructs where a single point mutation was made and a deltaG is assigned based on that mutation.

[SHEPHARD track files](https://shephard.readthedocs.io/en/latest/shephard_file_types.html#track-files) contain at least 3 columns:

* Column 1: protein identifier
* Column 2: track name
* Column 3 onwards: per-amino acid scores 

For the SHEPHARD files here, the track names are written as `deltaG_<X>` where `<X>` is an amino acid. The deltaGs are then the values where the residue at each position is mutated to amino acid `<X>`.

For example, if your had a sequence that was:

    >my_protein
    MAPL...
    
You might have a corresponding deltaG file with the format:

    my_protein    deltaG_A    2.3    0    5.6    1.2    ... 

i.e. for `my_protein` the ΔG reported based on the combined digestion by tyrpsin and chymotrypsin for the M1A mutation is 2.3 kcal/mol, whereas for P3A the ΔG is 5.6 kcal/mol.


## Usage
This notebook could be edited to extract additional columns. To keep things easy to understand, we have avoided using any third-party libraries (pandas, numpy etc) so you can read the file parsing in pure Python. This script does require [protfasta](https://protfasta.readthedocs.io/) to write out the sequence FASTA file, although you could also write this out without protfasta should you chose.

**You don't actually need to re-run this notebook** - we've just included it to clearly show how the Track files where generated.

For an example of how the data can be used see the demo notebooks in the upper directory.

## More details
The files here work with [SHEPHARD](https://www.biorxiv.org/content/10.1101/2022.09.18.508433v1). Documentation for SHEPHARD [is available here](https://shephard.readthedocs.io/).


## Reference
Tsuboyama, K., Dauparas, J., Chen, J., Mangan, N. M., Ovchinnikov, S., & Rocklin, G. J. (2022). Mega-scale experimental analysis of protein folding stability in biology and protein design. In bioRxiv (p. 2022.12.06.519132). https://doi.org/10.1101/2022.12.06.519132

### Read in data and print the header-to-index mapping

In [ ]:
import protfasta
import os
VALID_AMINO_ACIDS = ['A',  'C',  'D',  'E',  'F',  'G',  'H',  'I',  'K',  'L',  'M',  'N',  'P',  'Q',  'R',  'S',  'T',  'V',  'W',  'Y']



In [ ]:
# note - make sure this points at te CSV file downloaded from https://zenodo.org/record/7401275
with open('K50_dG_Dataset1_Dataset2.csv','r') as fh:
    content = fh.readlines()

# for the first line, remove the trailing newline and split on commas into a list
headers = content[0].strip().split(',')    

# finally print out the index-to-header mapping for completeness
print('Printing mapping of index to header name below:')
print('------------------------------------------------')
for idx, name in enumerate(headers):
    print(f'{idx}: {name}')

### Main data construction loop
The loop below loops over every line the input file and extracts out data for the associated mutation. It limits this ONLY to data records where a single point mutation was made.

In [ ]:
# initialize some stuff
muts = []
names2data = {}
names2seq  = {}
names2seq_full  = {}

# cycle through each data line in the file
for line in content[1:]:
    
    # extract out columns
    sline = line.strip().split(',')
    
    # read data from columns
    name = sline[29]
    seq_full  = sline[26]
    seq  = sline[27]
    mut  = sline[28]
    seq_len = len(seq)
    
    # if this is a wildtype sequence assign
    if mut == 'wt':
        
        # sanity check 
        if name in names2seq:
            if names2seq[name] != seq:
                raise Exception('Expected wildtype sequences to match but they don')
                
        
        names2seq[name] = seq
        names2seq_full[name] = seq_full
        continue
    else:
        
        # this try/except loop will fail if the sequence isn't
        # of format X<POSITION>Y where X and Y are single-letter
        # codes for valid amino acids and POSITION isn't a int
        # convertable string - i.e. this ensures we only extract
        # out mutations that are single point non-synonymous mutations
        try:
            pos = int(mut[1:-1])
            r1 = mut[0]
            r2 = mut[-1]
        except Exception:
            continue
    
    # define strings for the variables we care aboute!
    deltaG_t    = f'{float(sline[8]):1.3f}'
    deltaG_t_CI = f'{float(sline[11]):1.3f}'

    deltaG_c    = f'{float(sline[18]):1.3f}'
    deltaG_c_CI = f'{float(sline[21]):1.3f}'

    deltaG    = f'{float(sline[22]):1.3f}'
    deltaG_CI = f'{float(sline[25]):1.3f}'
    
    log10_K50_c    = f'{float(sline[12]):1.3f}'
    log10_K50_c_CI = f'{float(sline[15]):1.3f}'
    
    log10_K50_t    = f'{float(sline[2]):1.3f}'
    log10_K50_t_CI =f'{float(sline[5]):1.3f}'
        
    # the 'seq' variable here is the sequence of the mutant protein
    # sequence, so ensure the residue at position 'pos' in the mutant
    # is the same as residue Y in X<POS>Y 
    if seq[pos-1] != r2:        
        raise Exception(f'Expected amino acid at position {pos-1} to be {r2} but was actually {seq[pos-1]}')        
    
    # if this is the first time we encounter this protein
    if name not in names2data:
        names2data[name] = []
    
    # to the protein ID entry in names2data append a list
    # with the data elements defined below. 
    names2data[name].append([r1,pos,r2,deltaG_t, deltaG_t_CI, deltaG_c, deltaG_c_CI, deltaG, deltaG_CI, log10_K50_c, log10_K50_c_CI, log10_K50_t, log10_K50_t_CI])
        

In [ ]:
# Sanity check:
# Ensure wildtype sequence matches expectation

for name in names2seq:
    seq = names2seq[name]
    
    for entry in names2data[name]:
        if entry[0] != seq[entry[1]-1]:
            raise Exception(f'Expected residue {entry[1]} in sequence to match the X in X<to>Y but did not')

print('Sanity check complete')            

In [ ]:
# ..................................................
#
def rm(filename):
    """
    Function that removes a file if found (and ignores if 
    not).
    
    Parameters
    --------------
    filename : str
        Name of the file to be removed
        
    Returns
    -------------
    None
    """
    try:
        os.remove(filename)
    except FileNotFoundError:
        pass

# ..................................................
#
def write_file(filename, outdict, prefix):
    """
    Function that writes out a SHEPHARD-compliant 
    track file based on the passed filename and a 
    dictionary where each key-value pair in the
    dictionary has a key that is one of the 20 standard
    amino acids and the value is a list with values
    associated with position as long the sequence
    where the residue was mutated to the key amino
    acid.
    
    This is just a helper function for writing track
    files.
    
    Parameters
    --------------
    filename : str
        Output filename (note this will be appended to),
        as opposed to overwritten!
        
    outdict : dict
        Dictionary mapping amino acids to per-residue 
        values where each value is represent as a string

    Returns
    -------------
    None
    """

    with open(filename,'a') as fh:
        for  aa in VALID_AMINO_ACIDS:
            line = name + "\t" + f"{prefix}_{aa}"
            for val in outdict[aa]:
                line = line + '\t' + val

            line = line + '\n'
            fh.write(line)

In [ ]:
# Finally, write our TRACK files out
# remove existing files because we're gonna append

rm('shprd_track_deltaG.tsv')
rm('shprd_track_deltaG_CI.tsv')

rm('shprd_track_deltaG_t.tsv')
rm('shprd_track_deltaG_CI.tsv')

rm('shprd_track_deltaG_c_CI.tsv')
rm('shprd_track_deltaG_t_CI.tsv')

rm('shprd_track_log10_K50_c.tsv')
rm('shprd_track_log10_K50_c_CI.tsv')

rm('shprd_track_log10_K50_t.tsv')
rm('shprd_track_log10_K50_t_CI.tsv')


# write everything out
for name in names2seq:
    local_deltaG_t = {}
    local_deltaG_t_CI = {}

    local_deltaG_c = {}
    local_deltaG_c_CI = {}
    
    local_deltaG = {}
    local_deltaG_CI = {}
    
    local_log10_K50_t = {}
    local_log10_K50_t_CI = {}

    local_log10_K50_c = {}
    local_log10_K50_c_CI = {}
    
    for aa in VALID_AMINO_ACIDS:
        
        local_deltaG_t[aa] = ['0']*len(names2seq[name])
        local_deltaG_t_CI[aa] = ['0']*len(names2seq[name])

        local_deltaG_c[aa] = ['0']*len(names2seq[name])
        local_deltaG_c_CI[aa] = ['0']*len(names2seq[name])
    
        local_deltaG[aa] = ['0']*len(names2seq[name])
        local_deltaG_CI[aa] = ['0']*len(names2seq[name])

        local_log10_K50_t[aa] = ['0']*len(names2seq[name])
        local_log10_K50_t_CI[aa] = ['0']*len(names2seq[name])        
        
        local_log10_K50_c[aa] = ['0']*len(names2seq[name])
        local_log10_K50_c_CI[aa] = ['0']*len(names2seq[name])
        
        
    for entry in names2data[name]:
        pos = entry[1]-1
        r2 = entry[2]
        
        deltaG_t    = entry[3]
        deltaG_t_CI = entry[4]

        deltaG_c    = entry[5]
        deltaG_c_CI = entry[6]

        deltaG      = entry[7]
        deltaG_CI   = entry[8]
        
        log10_K50_c    = entry[9]
        log10_K50_c_CI = entry[10]
    
        log10_K50_t    = entry[11]
        log10_K50_t_CI = entry[12]
        
        
        local_deltaG[r2][pos] = deltaG
        local_deltaG_CI[r2][pos] = deltaG_CI
        
        local_deltaG_t[r2][pos] = deltaG_t
        local_deltaG_t_CI[r2][pos] = deltaG_t_CI
        
        local_deltaG_c[r2][pos] = deltaG_c
        local_deltaG_c_CI[r2][pos] = deltaG_c_CI
        
        
        local_log10_K50_t[r2][pos] = log10_K50_c
        local_log10_K50_t_CI[r2][pos] = log10_K50_c_CI
        
        local_log10_K50_c[r2][pos] = log10_K50_t
        local_log10_K50_c_CI[r2][pos] = log10_K50_t_CI

    write_file('shprd_track_deltaG.tsv', local_deltaG, 'deltaG')
    write_file('shprd_track_deltaG_CI.tsv', local_deltaG_CI, 'deltaG_CI')
    
    write_file('shprd_track_deltaG_t.tsv', local_deltaG_t, 'deltaG_t')
    write_file('shprd_track_deltaG_t_CI.tsv', local_deltaG_t_CI, 'deltaG_t_CI')
    
    write_file('shprd_track_deltaG_c.tsv', local_deltaG_c, 'deltaG_c')
    write_file('shprd_track_deltaG_c_CI.tsv', local_deltaG_c_CI, 'deltaG_c_CI')

    write_file('shprd_track_log10_K50_c.tsv', local_log10_K50_t, 'log10_K50_t')
    write_file('shprd_track_log10_K50_c_CI.tsv', local_log10_K50_t_CI, 'log10_K50_t_CI')
              
    write_file('shprd_track_log10_K50_t.tsv', local_log10_K50_c, 'log10_K50_c')
    write_file('shprd_track_log10_K50_t_CI.tsv', local_log10_K50_c_CI, 'log10_K50_c_CI')
            
        

In [ ]:
# Finally write a FASTA file out with the sequences parsed in this notebook
protfasta.write_fasta(names2seq, 'sequences.fasta')